Capstone Project Part 1

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [4]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


Part 1: Scraping Data

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get(url).text

In [6]:
soup = BeautifulSoup(html_data, "html5lib")

In [7]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
neighborhoods_df=pd.DataFrame(table_contents)
neighborhoods_df['Borough']=neighborhoods_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [8]:
neighborhoods_df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Part 2: Adding lat and long values

In [9]:
latitude=43.6511
longitude=-79.38

Uploaded the CSV to my project, used the "find and add data" button on the top to spit out this code

In [10]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Used Inner Join

In [11]:
neighborhoods = pd.merge(left=neighborhoods_df, right=df_data_1, left_on='Postal Code', right_on='Postal Code')

In [12]:
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


Part 3: Clustering

Fun with Maps

In [13]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
done

# All requested packages already installed.



In [14]:
import folium

In [15]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [16]:
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

In [17]:
map_toronto

Examining Nearby Venues

In [18]:
nearby_venues1 = pd.read_json("https://raw.githubusercontent.com/ibm-developer-skills-network/yczvh-DataFilesForIBMProjects/master/segmenting_neighborhoods.json")    
nearby_venues1.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                 'Venue', 
                 'Venue Latitude', 
                 'Venue Longitude', 
                 'Venue Category']
Toronto_venues=nearby_venues1

In [19]:
Toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
...,...,...,...,...,...,...,...
1332,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,LCBO,43.741508,-79.584501,Liquor Store
1333,"Clairville, Humberwood, Woodbine Downs, West H...",43.706748,-79.594054,Economy Rent A Car,43.708471,-79.589943,Rental Car Location
1334,"Clairville, Humberwood, Woodbine Downs, West H...",43.706748,-79.594054,Logistics Distribution,43.707554,-79.589252,Bar
1335,"Clairville, Humberwood, Woodbine Downs, West H...",43.706748,-79.594054,Saand Rexdale,43.705072,-79.598725,Drugstore


In [20]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
...,...,...,...,...,...,...
Willowdale West,5,5,5,5,5,5
"Willowdale, Newtonbrook",1,1,1,1,1,1
Woburn,4,4,4,4,4,4


In [21]:
one_hot = pd.get_dummies(Toronto_venues, columns = ['Venue Category'])

Group Venues by Neighborhood

In [24]:
toronto_grouped = one_hot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Venue Category_Accessories Store,Venue Category_Adult Boutique,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,...,Venue Category_Train Station,Venue Category_Truck Stop,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Video Game Store,Venue Category_Vietnamese Restaurant,Venue Category_Warehouse Store,Venue Category_Wine Bar,Venue Category_Wings Joint,Venue Category_Women's Store,Venue Category_Yoga Studio
0,Agincourt,43.794200,-79.262029,43.792971,-79.261496,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",43.602414,-79.543484,43.601580,-79.545312,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,43.755753,-79.440219,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,43.786947,-79.385975,43.787903,-79.380860,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",43.733282,-79.419750,43.733805,-79.419393,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Willowdale West,43.782736,-79.442259,43.783460,-79.445736,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,"Willowdale, Newtonbrook",43.789053,-79.408493,43.786942,-79.410022,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,Woburn,43.770992,-79.216917,43.769837,-79.219237,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,43.695344,-79.318389,43.696130,-79.315663,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Finding Top Venues for Each Neighborhood (had to drop latitude values because they were the highest numbers and coming up as top venues)

In [26]:
num_top_venues = 5

In [27]:
toronto_temp = toronto_grouped.drop("Neighborhood Latitude", axis=1).drop("Venue Latitude", axis = 1)
toronto_temp

,Neighborhood,Neighborhood Longitude,Venue Longitude,Venue Category_Accessories Store,Venue Category_Adult Boutique,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,...,Venue Category_Train Station,Venue Category_Truck Stop,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Video Game Store,Venue Category_Vietnamese Restaurant,Venue Category_Warehouse Store,Venue Category_Wine Bar,Venue Category_Wings Joint,Venue Category_Women's Store,Venue Category_Yoga Studio
0,Agincourt,-79.262029,-79.261496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",-79.543484,-79.545312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",-79.442259,-79.440219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,-79.385975,-79.380860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",-79.419750,-79.419393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Willowdale West,-79.442259,-79.445736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,"Willowdale, Newtonbrook",-79.408493,-79.410022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,Woburn,-79.216917,-79.219237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,-79.318389,-79.315663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
for hood in toronto_temp['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_temp[toronto_temp['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                                      venue  freq
0                     Venue Category_Lounge  0.25
1  Venue Category_Latin American Restaurant  0.25
2             Venue Category_Breakfast Spot  0.25
3               Venue Category_Skating Rink  0.25
4              Venue Category_Grocery Store  0.00


----Alderwood, Long Branch----
                           venue  freq
0     Venue Category_Pizza Place  0.25
1     Venue Category_Coffee Shop  0.12
2    Venue Category_Skating Rink  0.12
3  Venue Category_Sandwich Place  0.12
4             Venue Category_Pub  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
                                      venue  freq
0                       Venue Category_Bank  0.09
1                Venue Category_Coffee Shop  0.09
2                Venue Category_Pizza Place  0.04
3                 Venue Category_Restaurant  0.04
4  Venue Category_Middle Eastern Restaurant  0.04


----Bayview Village----
                               

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_temp.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Venue Category_Lounge,Venue Category_Latin American Restaurant,Venue Category_Breakfast Spot,Venue Category_Skating Rink,Venue Category_Grocery Store,Venue Category_Martial Arts School,Venue Category_Medical Center,Venue Category_Mediterranean Restaurant,Venue Category_Men's Store,Venue Category_Metro Station
1,"Alderwood, Long Branch",Venue Category_Pizza Place,Venue Category_Coffee Shop,Venue Category_Skating Rink,Venue Category_Sandwich Place,Venue Category_Pub,Venue Category_Pharmacy,Venue Category_Gym,Venue Category_Park,Venue Category_Organic Grocery,Venue Category_Massage Studio
2,"Bathurst Manor, Wilson Heights, Downsview North",Venue Category_Bank,Venue Category_Coffee Shop,Venue Category_Pizza Place,Venue Category_Restaurant,Venue Category_Middle Eastern Restaurant,Venue Category_Pharmacy,Venue Category_Shopping Mall,Venue Category_Mobile Phone Shop,Venue Category_Sandwich Place,Venue Category_Supermarket
3,Bayview Village,Venue Category_Bank,Venue Category_Café,Venue Category_Chinese Restaurant,Venue Category_Japanese Restaurant,Venue Category_Mobile Phone Shop,Venue Category_Movie Theater,Venue Category_Motel,Venue Category_Monument / Landmark,Venue Category_Modern European Restaurant,Venue Category_Grocery Store
4,"Bedford Park, Lawrence Manor East",Venue Category_Italian Restaurant,Venue Category_Coffee Shop,Venue Category_Sandwich Place,Venue Category_Grocery Store,Venue Category_Comfort Food Restaurant,Venue Category_Cupcake Shop,Venue Category_Pub,Venue Category_Restaurant,Venue Category_Café,Venue Category_Butcher


In [31]:
neighborhoods_venues_sorted.shape

(100, 11)

Fun with K Means

In [32]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [33]:
from sklearn.cluster import KMeans

In [34]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 1, 1, 0, 1, 1, 3], dtype=int32)

In [95]:
toronto_merged = neighborhoods

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(10) # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Venue Category_Park,Venue Category_Food & Drink Shop,Venue Category_Grocery Store,Venue Category_Motel,Venue Category_Martial Arts School,Venue Category_Massage Studio,Venue Category_Medical Center,Venue Category_Mediterranean Restaurant,Venue Category_Men's Store,Venue Category_Metro Station
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Venue Category_Hockey Arena,Venue Category_Coffee Shop,Venue Category_Intersection,Venue Category_Portuguese Restaurant,Venue Category_French Restaurant,Venue Category_Pizza Place,Venue Category_Mediterranean Restaurant,Venue Category_Men's Store,Venue Category_Metro Station,Venue Category_Mexican Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Venue Category_Coffee Shop,Venue Category_Park,Venue Category_Café,Venue Category_Bakery,Venue Category_Breakfast Spot,Venue Category_Yoga Studio,Venue Category_Spa,Venue Category_Mexican Restaurant,Venue Category_French Restaurant,Venue Category_Distribution Center
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Venue Category_Clothing Store,Venue Category_Furniture / Home Store,Venue Category_Accessories Store,Venue Category_Boutique,Venue Category_Vietnamese Restaurant,Venue Category_Coffee Shop,Venue Category_Miscellaneous Shop,Venue Category_Movie Theater,Venue Category_Motel,Venue Category_Monument / Landmark
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1.0,Venue Category_Coffee Shop,Venue Category_Sushi Restaurant,Venue Category_Diner,Venue Category_Yoga Studio,Venue Category_Theater,Venue Category_Burrito Place,Venue Category_Café,Venue Category_Beer Bar,Venue Category_Sandwich Place,Venue Category_Fried Chicken Joint
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Venue Category_Fast Food Restaurant,Venue Category_Grocery Store,Venue Category_Motel,Venue Category_Martial Arts School,Venue Category_Massage Studio,Venue Category_Medical Center,Venue Category_Mediterranean Restaurant,Venue Category_Men's Store,Venue Category_Metro Station,Venue Category_Mexican Restaurant
7,M3B,North York,Don Mills North,43.745906,-79.352188,1.0,Venue Category_Gym,Venue Category_Café,Venue Category_Construction & Landscaping,Venue Category_Caribbean Restaurant,Venue Category_Japanese Restaurant,Venue Category_Modern European Restaurant,Venue Category_Movie Theater,Venue Category_Motel,Venue Category_Monument / Landmark,Venue Category_Miscellaneous Shop
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,0.0,Venue Category_Pizza Place,Venue Category_Athletics & Sports,Venue Category_Gym / Fitness Center,Venue Category_Pharmacy,Venue Category_Breakfast Spot,Venue Category_Bank,Venue Category_Flea Market,Venue Category_Café,Venue Category_Intersection,Venue Category_Gastropub
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Venue Category_Café,Venue Category_Clothing Store,Venue Category_Theater,Venue Category_Pizza Place,Venue Category_Electronics Store,Venue Category_Coffee Shop,Venue Category_Burger Joint,Venue Category_Music Venue,Venue Category_Bakery,Venue Category_Steakhouse


In [39]:
import matplotlib.cm as cm
import matplotlib.colors as colors

Quick Clean Up

In [88]:
toronto_merged.dropna(inplace=True)

toronto_merged.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Venue Category_Park,Venue Category_Food & Drink Shop,Venue Category_Grocery Store,Venue Category_Motel,Venue Category_Martial Arts School,Venue Category_Massage Studio,Venue Category_Medical Center,Venue Category_Mediterranean Restaurant,Venue Category_Men's Store,Venue Category_Metro Station
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Venue Category_Hockey Arena,Venue Category_Coffee Shop,Venue Category_Intersection,Venue Category_Portuguese Restaurant,Venue Category_French Restaurant,Venue Category_Pizza Place,Venue Category_Mediterranean Restaurant,Venue Category_Men's Store,Venue Category_Metro Station,Venue Category_Mexican Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Venue Category_Coffee Shop,Venue Category_Park,Venue Category_Café,Venue Category_Bakery,Venue Category_Breakfast Spot,Venue Category_Yoga Studio,Venue Category_Spa,Venue Category_Mexican Restaurant,Venue Category_French Restaurant,Venue Category_Distribution Center
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Venue Category_Clothing Store,Venue Category_Furniture / Home Store,Venue Category_Accessories Store,Venue Category_Boutique,Venue Category_Vietnamese Restaurant,Venue Category_Coffee Shop,Venue Category_Miscellaneous Shop,Venue Category_Movie Theater,Venue Category_Motel,Venue Category_Monument / Landmark
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1.0,Venue Category_Coffee Shop,Venue Category_Sushi Restaurant,Venue Category_Diner,Venue Category_Yoga Studio,Venue Category_Theater,Venue Category_Burrito Place,Venue Category_Café,Venue Category_Beer Bar,Venue Category_Sandwich Place,Venue Category_Fried Chicken Joint
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Venue Category_Fast Food Restaurant,Venue Category_Grocery Store,Venue Category_Motel,Venue Category_Martial Arts School,Venue Category_Massage Studio,Venue Category_Medical Center,Venue Category_Mediterranean Restaurant,Venue Category_Men's Store,Venue Category_Metro Station,Venue Category_Mexican Restaurant
7,M3B,North York,Don Mills North,43.745906,-79.352188,1.0,Venue Category_Gym,Venue Category_Café,Venue Category_Construction & Landscaping,Venue Category_Caribbean Restaurant,Venue Category_Japanese Restaurant,Venue Category_Modern European Restaurant,Venue Category_Movie Theater,Venue Category_Motel,Venue Category_Monument / Landmark,Venue Category_Miscellaneous Shop
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,0.0,Venue Category_Pizza Place,Venue Category_Athletics & Sports,Venue Category_Gym / Fitness Center,Venue Category_Pharmacy,Venue Category_Breakfast Spot,Venue Category_Bank,Venue Category_Flea Market,Venue Category_Café,Venue Category_Intersection,Venue Category_Gastropub
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Venue Category_Café,Venue Category_Clothing Store,Venue Category_Theater,Venue Category_Pizza Place,Venue Category_Electronics Store,Venue Category_Coffee Shop,Venue Category_Burger Joint,Venue Category_Music Venue,Venue Category_Bakery,Venue Category_Steakhouse
10,M6B,North York,Glencairn,43.709577,-79.445073,1.0,Venue Category_Pizza Place,Venue Category_Sushi Restaurant,Venue Category_Bakery,Venue Category_Japanese Restaurant,Venue Category_Grocery Store,Venue Category_Monument / Landmark,Venue Category_Massage Studio,Venue Category_Medical Center,Venue Category_Mediterranean Restaurant,Venue Category_Men's Store


In [92]:
toronto_merged = toronto_merged.astype({'Cluster Labels': 'int'})

Fun with maps, part 2

In [93]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters